# Tutorial 2: Tokenizadores, Stemming, Lemmatization y WordClouds.

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Fabian Villena](https://fabianvillena.cl/).
- Profesora Auxiliar: María José Zambrano



## 1 Pre-procesamiento

In [ ]:
!pip install datasets

In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
!pip install tokenizers

In [ ]:
!pip install nltk

In [ ]:
!pip install matplotlib

In [6]:
import datasets
import pandas as pd
import re
import tokenizers
import nltk



In [ ]:
nltk.download('omw-1.4')

In [ ]:
nltk.download('punkt_tab')


## 🤗 Datasets

🤗 (HuggingFace) Datasets es una biblioteca de manejo de conjuntos de datos para procesamiento de lenguaje natural que se destaca por la simplicidad de sus métodos y el gran repositorio 🤗 Hub que contiene muchos conjuntos de datos libres para descargar sólo con una linea de Python.

En nuestro curso trabajaremos con `spanish_diagnostics`, un conjunto de datos de nuestro grupo investigación PLN@CMM que contiene textos de sospechas diagnósticas de la lista de espera chilena y está etiquetado con el destino de la interconsulta; este destino puede ser `dental` o `no_dental`.

In [ ]:
spanish_diagnostics = datasets.load_dataset('fvillena/spanish_diagnostics') # Con esta linea descargamos el conjunto de datos completo

In [ ]:
spanish_diagnostics

In [ ]:
spanish_diagnostics['train'][0]

In [12]:
def normalize(text, remove_tildes = True):
    """Normaliza una cadena de texto convirtiéndo todo a minúsculas, quitando los caracteres no alfabéticos y los tildes"""
    text = text.lower() # Llevamos todo a minúscula
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) # Reemplazamos los caracteres no alfabéticos por un espacio
    if remove_tildes:
        text = re.sub('á', 'a', text) # Reemplazamos los tildes
        text = re.sub('é', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ó', 'o', text)
        text = re.sub('ú', 'u', text)
    return text

In [ ]:
spanish_diagnostics_normalized = spanish_diagnostics["train"].map(
    lambda x: { # Utilizamos una función anónima que devuelve un diccionario
        "normalized_text" : normalize(x["text"]) # Esta es una nueva característica que agregaremos a nuestro conjunto de datos.
    })

In [ ]:
spanish_diagnostics_normalized

Ahora nuestro conjunto de datos cuenta con una nueva característica `normalized_text`.

In [ ]:
spanish_diagnostics_normalized[0]

## Tokenización

La tokenización es el proceso de demarcación de secciones de una cadena de caracteres. Estas secciones podrían ser oraciones, palabras o subpalabras.

El método más simple para tokenizar una cadena de caracteres en nuestro lenguaje es la separación por espacios. Aplicamos una separación por espacios mediante el método `str.split()` sobre nuestro conjunto de datos normalizado.

In [ ]:
spanish_diagnostics_normalized[0]["normalized_text"]

In [ ]:
spanish_diagnostics_normalized[0]["normalized_text"].split()

Si bien el método de separación por espacios funciona bien en nuestro conjunto de datos normalizado, también quisiéramos tokenizar nuestro texto sin normalizar.

In [ ]:
spanish_diagnostics_normalized[0]["text"]

In [ ]:
spanish_diagnostics_normalized[0]["text"].split()

In [20]:
# Using spacy.load().
import spacy
nlp = spacy.load("es_core_news_sm")

Al aplicar el mismo métodos podemos observar que no funciona totalmente bien debido a la presencia de caracteres no alfabéticos. Para solucionar esto, existen métodos basados en una serie de reglas para solucionar estos problemas. Utilizaremos la implementación de un tokenizador basado en reglas de la biblioteca de procesamiento de lenguaje natural Spacy.

In [21]:
spacy_tokenizer = nlp.tokenizer

In [ ]:
list(spacy_tokenizer(spanish_diagnostics_normalized[0]["text"]))

Al utilizar el tokenizador basado en reglas, podemos tener resultados mucho mejores que los anteriores.

### Tokenizadores 👹

🤗 también cuenta con una biblioteca llamada Tokenizers, con la cual podemos construir nuestro tokenizador basado en nuestro conjunto de datos.

Instanciamos el tokenizador con un modelo WordPiece, el cual parte construyendo un vocabulario que incluye todas los caracteres presentes en el conjunto de datos y posteriormente comienza a mezclar caracteres hasta encontrar conjuntos de caracteres que tienen más probabilidad de aparecer juntos que separados.

In [23]:
tokenizer = tokenizers.Tokenizer(tokenizers.models.WordPiece())

Esta biblioteca nos permite añadir pasos de normalización directamente. Replicamos lo mismo que hacemos con nuestra función `normalizer()`.

In [24]:
normalizer = tokenizers.normalizers.Sequence([
    tokenizers.normalizers.Lowercase(), # Llevamos todo a minúscula
    tokenizers.normalizers.NFD(), # Separamos cada caracter según los elementos que lo componen: á -> (a, ´)
    tokenizers.normalizers.StripAccents(), # Eliminamos todos los acentos
    tokenizers.normalizers.Replace(tokenizers.Regex(r"[^a-z ]"), " ") # Reemplazamos todos los caracteres no alfabéticos
])

In [ ]:
normalizer.normalize_str(spanish_diagnostics_normalized[0]["text"])

Añadimos el normalizador al tokenizador

In [26]:
tokenizer.normalizer = normalizer

Pre tokenizamos nuestro conjunto de datos mediante espacio para delimitar el tamaño que puede tener cada token.

In [27]:
tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Whitespace()

Instanciamos el entrenador que entrenará nuestro tokenizador.

In [28]:
trainer = tokenizers.trainers.WordPieceTrainer()

Entrenamos el tokenizador sobre nuestro conjunto de datos.

In [29]:
tokenizer.train_from_iterator(spanish_diagnostics_normalized["text"], trainer)

Mediante el método `Tokenizer.encode()` obtenemos la representación tokenizada de nuesto texto. Esta representación contiene varios atributos, donde los más interesantes son:

- `ids`: Contiene nuestro texto representado a través de una lista que contiene los identificadores de cada token.
- `tokens`: Contiene nuestro texto representado a través de una lista que contiene el texto de cada token.

In [ ]:
spanish_diagnostics_normalized[0]["text"]

In [31]:
tokenized_output = tokenizer.encode(spanish_diagnostics_normalized[0]["text"])

In [32]:
#para ver todo el vocabulario del tonekizador consultar la siguiente funcion
#tokenizer.get_vocab()

In [ ]:
tokenized_output.ids

In [ ]:
tokenized_output.tokens

Tal como lo hicimos anteriormente podemos aplicar paralelamente nuestro tokenizador sobre el conjunto de datos mediante el método `Dataset.map()`

In [ ]:
spanish_diagnostics_normalized_tokenized = spanish_diagnostics_normalized.map(lambda x: {"tokenized_text":tokenizer.encode(x["text"]).tokens})

Nuestro conjunto de datos ahora contiene el texto tokenizado en la característica `tokenized_text`.

In [ ]:
spanish_diagnostics_normalized_tokenized[0]

## Stemming y Lematización

Con el fin de disminuir la cantidad de características de las representaciones de texto existen métodos que reducen el tamaño de vocabulario al eliminar inflexiones que puedan tener las palabras. Estos métodos son:

- Lematización: Este método lleva una palabra en su forma flexionada a su forma base, por ejemplo *tratada* -> *tratar*
- Stemming: Este método trunca las palabras de entrada mediante un algoritmo predefinido para encontrar la raíz de la misma, por ejemplo *tratada* -> *trat*

El proceso de lematización lo haremos a través de la biblioteca Spacy y el proceso de stemming a través de la biblioteca NLTK utilizando el algoritmo Snowball.

Instanciamos el analizador de Spacy

Definimos como tokenizador el que entrenamos anteriormente.

In [37]:
def custom_tokenizer(text):
    tokens = tokenizer.encode(text).tokens
    return spacy.tokens.Doc(nlp.vocab,tokens)

In [38]:
nlp.tokenizer = custom_tokenizer

Instanciamos el Stemmer

In [39]:
stemmer = nltk.stem.SnowballStemmer("spanish")

Podemos verificar cómo funcionan estos métodos sobre un texto de prueba de nuestro conjunto de datos.

In [ ]:
spanish_diagnostics_normalized_tokenized[5]["text"]

In [ ]:
for t in nlp(spanish_diagnostics_normalized_tokenized[5]["text"]):
    print(f"Token: {t.text}\nLema: {t.lemma_}\nStem: {stemmer.stem(t.text)}\n---")

# Pre-procesamiento con NLTK

En Python existe en "Natural Language Toolkit" que tiene un [libro asociado](http://www.nltk.org/book/). Ejemplos de tokenizadores mas comunes de la libreria (Natural Language Toolkit) :

- Whitespace tokenization: Este enfoque simple divide una cadena de texto en tokens utilizando los espacios en blanco como delimitadores. Cada palabra o elemento separado por espacios en blanco se considera un token. Por ejemplo, la cadena "Hola, ¿cómo estás?" se dividiría en los tokens "Hola,", "¿cómo" y "estás?".

- Punctuation-based tokenization: Este tokenizador se basa en los signos de puntuación para dividir una cadena de texto en tokens. Cada signo de puntuación se considera un token independiente. Por ejemplo, la cadena "¡Hola! ¿Cómo estás?" se dividiría en los tokens "¡", "Hola", "!", "¿", "Cómo" y "estás".

- Default/TreebankWordTokenizer: Este tokenizador se basa en las convenciones utilizadas en el corpus de Treebank del Penn Treebank. Divide una cadena de texto en tokens siguiendo reglas específicas, como separar los signos de puntuación adyacentes de las palabras. Por ejemplo, la cadena "I can't go" se dividiría en los tokens "I", "ca", "n't" y "go".

- TweetTokenizer: Este tokenizador está diseñado específicamente para manejar texto de redes sociales, como tweets. Tiene en cuenta las convenciones y peculiaridades del lenguaje utilizado en las redes sociales, como hashtags, menciones de usuarios, emoticones, etc. Por ejemplo, la cadena "I love #NLTK!" se dividiría en los tokens "I", "love", "#NLTK" y "! ".

- MWETokenizer: Este tokenizador se utiliza para identificar y tratar expresiones compuestas por múltiples palabras como un solo token. Por ejemplo, en lugar de dividir la expresión "New York City" en tres tokens separados, este tokenizador la mantendría intacta como un único token.

[![medium](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*5385R1NI5mNm2J4WSlXo6A.png)](https://towardsdatascience.com/top-5-word-tokenizers-that-every-nlp-data-scientist-should-know-45cc31f8e8b9)

In [42]:
import nltk

In [ ]:
nltk.download(['punkt','wordnet','gutenberg','webtext','stopwords'])

## Tokens

In [44]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [45]:
example = "Hola! Como están? Después de esta clase yo me voy a mi casa"

In [ ]:
print(sent_tokenize(example))

In [ ]:
print(word_tokenize(example))

## Stopwords

In [48]:
from nltk.corpus import stopwords

In [49]:
stop_words= set(stopwords.words('spanish'))

In [ ]:
stop_words

In [ ]:
wordTokens = word_tokenize(example)
print(wordTokens)

In [ ]:
filtered = [w for w in wordTokens if not w in stop_words]
print(filtered)

## Stemming vs. lemmatization (English)

In [53]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [54]:
exampleWords = ["love", "Loving","loved", "lovelii"]

In [ ]:
for w in exampleWords:
    print(ps.stem(w))

Lemmatization

In [56]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
for w in exampleWords:
    print(lemmatizer.lemmatize(w))

In [61]:
# It seems it didn't work! It just copied the words!

## WordNet

WordNet es un diccionario léxico que organiza las palabras en grupos de sinónimos llamados "synsets" (conjuntos de sinónimos). Cada synset representa un concepto léxico y contiene una lista de palabras o frases que son intercambiables en ciertos contextos. Además de los sinónimos, WordNet también proporciona información semántica sobre las relaciones entre los synsets, como hiperónimos (conceptos más generales) e hipónimos (conceptos más específicos).

In [58]:
from nltk.corpus import wordnet as wn

In [ ]:
wn.synset('dog.n.01').lemma_names('spa')

In [60]:
syns = wn.synsets("program")

In [ ]:
print(syns[0].name())

In [ ]:
print(syns[2].name())

In [ ]:
type(syns)

In [ ]:
print(syns[0].lemmas()[0].name())

In [ ]:
print(syns[0].definition())

In [ ]:
print(syns[0].examples())

In [ ]:
synonyms = []
antonyms = []

for syn in wn.synsets("good"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())

print(set(synonyms))

In [ ]:
print(set(antonyms))

## Corpora

El proyecto Gutenberg tiene libros en español http://www.gutenberg.org/ebooks/search/?query=spanish
Y pueden seguir estos pasos para bajar libros automáticamente: https://pypi.org/project/Gutenberg/

Hay muchos tutoriales en internet para aprender a usar NLTK, por ejemplo este: https://vprusso.github.io/blog/2018/natural-language-processing-python-1/

In [ ]:
print(nltk.corpus.gutenberg.fileids())


In [74]:
from nltk.text import Text
macbeth = Text(nltk.corpus.gutenberg.words('shakespeare-macbeth.txt'))

In [ ]:
#palabras
print(len(macbeth))


In [ ]:
macbeth.tokens

In [ ]:
# palabras unicas
print(len(set(macbeth)))

In [ ]:
macbeth.concordance("father")

In [ ]:
macbeth.dispersion_plot(["Macbeth", "King", "Lady"])

## Contar palabras

In [ ]:
fdist = nltk.FreqDist(macbeth)
fdist.plot(20, cumulative=False)

Eliminando caracteres no alfabéticos

In [ ]:
fdist_no_punc = nltk.FreqDist(dict((word, freq) for word, freq in fdist.items() if word.isalpha()))
fdist_no_punc.plot(20, cumulative=False, title="20 most common tokens (no punctuation)")

Eliminando _stopwords_

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
fdist_no_punc_no_stopwords = nltk.FreqDist(dict((word, freq) for word, freq in fdist.items() if word not in stopwords and word.isalpha()))
fdist_no_punc_no_stopwords.plot(20, cumulative=False, title="20 most common tokens (no stopwords or punctuation)")

## Nubes de palabras

In [ ]:
!pip install wordcloud

In [92]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [93]:
text = "El curso de estadistica y  probabilidades para el análsis de datos está super bueno!"

In [ ]:
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

Funcionó, pero igual es feo :P

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

Y se pueden hacer nubes con formas también: https://www.datacamp.com/community/tutorials/wordcloud-python.
Estas nubes son más interesantes con textos más grandes.

In [ ]:
macbeth

In [ ]:
palabras = nltk.corpus.gutenberg.raw('shakespeare-macbeth.txt')
palabras

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(palabras)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

Y no sólo hay libros!

In [ ]:
for file_id in nltk.corpus.webtext.fileids():
    print(file_id)